In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 9)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [23, 25, 27]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.6581971645355225, Final Batch Loss: 2.3276689052581787
Epoch 2, Loss: 4.636362552642822, Final Batch Loss: 2.3187096118927
Epoch 3, Loss: 4.621701717376709, Final Batch Loss: 2.312567710876465
Epoch 4, Loss: 4.596620321273804, Final Batch Loss: 2.298973560333252
Epoch 5, Loss: 4.567766427993774, Final Batch Loss: 2.275578022003174
Epoch 6, Loss: 4.546075820922852, Final Batch Loss: 2.2594542503356934
Epoch 7, Loss: 4.528770446777344, Final Batch Loss: 2.26294207572937
Epoch 8, Loss: 4.51008415222168, Final Batch Loss: 2.2530200481414795
Epoch 9, Loss: 4.481331825256348, Final Batch Loss: 2.2376646995544434
Epoch 10, Loss: 4.456157922744751, Final Batch Loss: 2.213451623916626
Epoch 11, Loss: 4.436905860900879, Final Batch Loss: 2.215338706970215
Epoch 12, Loss: 4.410337924957275, Final Batch Loss: 2.210038185119629
Epoch 13, Loss: 4.387431621551514, Final Batch Loss: 2.1917552947998047
Epoch 14, Loss: 4.3548924922943115, Final Batch Loss: 2.163865089416504
Epoch 15, Lo

Epoch 117, Loss: 0.41146180033683777, Final Batch Loss: 0.20727378129959106
Epoch 118, Loss: 0.42499756813049316, Final Batch Loss: 0.20774629712104797
Epoch 119, Loss: 0.41605985164642334, Final Batch Loss: 0.20761975646018982
Epoch 120, Loss: 0.45146577060222626, Final Batch Loss: 0.22076736390590668
Epoch 121, Loss: 0.4088220000267029, Final Batch Loss: 0.20533987879753113
Epoch 122, Loss: 0.39419645071029663, Final Batch Loss: 0.19950512051582336
Epoch 123, Loss: 0.4453068971633911, Final Batch Loss: 0.2251034677028656
Epoch 124, Loss: 0.40099678933620453, Final Batch Loss: 0.20903867483139038
Epoch 125, Loss: 0.3641014099121094, Final Batch Loss: 0.16491347551345825
Epoch 126, Loss: 0.3813464045524597, Final Batch Loss: 0.21783749759197235
Epoch 127, Loss: 0.2928570359945297, Final Batch Loss: 0.14866508543491364
Epoch 128, Loss: 0.3168323189020157, Final Batch Loss: 0.1519295573234558
Epoch 129, Loss: 0.3635846823453903, Final Batch Loss: 0.1798103004693985
Epoch 130, Loss: 0.279

Epoch 248, Loss: 0.08290024660527706, Final Batch Loss: 0.061936136335134506
Epoch 249, Loss: 0.08900962956249714, Final Batch Loss: 0.06152114272117615
Epoch 250, Loss: 0.05852106213569641, Final Batch Loss: 0.020820636302232742
Epoch 251, Loss: 0.09347428008913994, Final Batch Loss: 0.043929025530815125
Epoch 252, Loss: 0.0715149324387312, Final Batch Loss: 0.023835795000195503
Epoch 253, Loss: 0.07694831117987633, Final Batch Loss: 0.03770304098725319
Epoch 254, Loss: 0.09392693266272545, Final Batch Loss: 0.058015406131744385
Epoch 255, Loss: 0.06345197930932045, Final Batch Loss: 0.04607947915792465
Epoch 256, Loss: 0.04766027256846428, Final Batch Loss: 0.014856413006782532
Epoch 257, Loss: 0.07063003256917, Final Batch Loss: 0.01806177943944931
Epoch 258, Loss: 0.06935320049524307, Final Batch Loss: 0.0379236601293087
Epoch 259, Loss: 0.08663555607199669, Final Batch Loss: 0.045857105404138565
Epoch 260, Loss: 0.075257976539433, Final Batch Loss: 0.014868863858282566
Epoch 261, 

Epoch 366, Loss: 0.02993207424879074, Final Batch Loss: 0.017631394788622856
Epoch 367, Loss: 0.0597545113414526, Final Batch Loss: 0.037126827985048294
Epoch 368, Loss: 0.04854267556220293, Final Batch Loss: 0.0349075123667717
Epoch 369, Loss: 0.028929652646183968, Final Batch Loss: 0.012928139418363571
Epoch 370, Loss: 0.06436705589294434, Final Batch Loss: 0.04223056137561798
Epoch 371, Loss: 0.04045131802558899, Final Batch Loss: 0.019295668229460716
Epoch 372, Loss: 0.031564013101160526, Final Batch Loss: 0.022963661700487137
Epoch 373, Loss: 0.03335757553577423, Final Batch Loss: 0.024269644170999527
Epoch 374, Loss: 0.07471923530101776, Final Batch Loss: 0.058873675763607025
Epoch 375, Loss: 0.03126103896647692, Final Batch Loss: 0.01361092273145914
Epoch 376, Loss: 0.02775638923048973, Final Batch Loss: 0.01690831035375595
Epoch 377, Loss: 0.07587878033518791, Final Batch Loss: 0.023822590708732605
Epoch 378, Loss: 0.057440485805273056, Final Batch Loss: 0.029237287119030952
Ep

Epoch 476, Loss: 0.028586280532181263, Final Batch Loss: 0.020018409937620163
Epoch 477, Loss: 0.008592871250584722, Final Batch Loss: 0.0036123262252658606
Epoch 478, Loss: 0.02958863298408687, Final Batch Loss: 0.0026814814191311598
Epoch 479, Loss: 0.017760268412530422, Final Batch Loss: 0.0018637096509337425
Epoch 480, Loss: 0.02718853624537587, Final Batch Loss: 0.023935772478580475
Epoch 481, Loss: 0.04741161596029997, Final Batch Loss: 0.03899746388196945
Epoch 482, Loss: 0.027006683871150017, Final Batch Loss: 0.009566999971866608
Epoch 483, Loss: 0.016847729682922363, Final Batch Loss: 0.011099434457719326
Epoch 484, Loss: 0.01022855518385768, Final Batch Loss: 0.005150030367076397
Epoch 485, Loss: 0.013764436356723309, Final Batch Loss: 0.006637799087911844
Epoch 486, Loss: 0.0408587115816772, Final Batch Loss: 0.00565590662881732
Epoch 487, Loss: 0.06640011630952358, Final Batch Loss: 0.04807175695896149
Epoch 488, Loss: 0.040547012351453304, Final Batch Loss: 0.009689646773

...

Epoch 4432, Loss: 0.00023173828958533704, Final Batch Loss: 3.5850549465976655e-05
Epoch 4433, Loss: 0.0001690099361439934, Final Batch Loss: 1.6685769878677092e-05
Epoch 4434, Loss: 0.00014316812666947953, Final Batch Loss: 9.208968549501151e-05
Epoch 4435, Loss: 0.00019433822490100283, Final Batch Loss: 0.00018695008475333452
Epoch 4436, Loss: 0.0005002115576644428, Final Batch Loss: 0.00043855392141267657
Epoch 4437, Loss: 0.00011492382691358216, Final Batch Loss: 3.393358565517701e-05
Epoch 4438, Loss: 0.00015759410143800778, Final Batch Loss: 6.7477258198778145e-06
Epoch 4439, Loss: 1.7644652871240396e-05, Final Batch Loss: 9.311563189839944e-06
Epoch 4440, Loss: 5.921859656154993e-05, Final Batch Loss: 5.059038358012913e-06
Epoch 4441, Loss: 0.004080950631760061, Final Batch Loss: 0.0003553625429049134
Epoch 4442, Loss: 7.921957512735389e-05, Final Batch Loss: 4.864096626988612e-05
Epoch 4443, Loss: 2.2856038413010538e-05, Final Batch Loss: 1.3347163076105062e-05
Epoch 4444, Loss

Epoch 4544, Loss: 6.001382007525535e-05, Final Batch Loss: 6.273138751566876e-06
Epoch 4545, Loss: 0.0009219957871664519, Final Batch Loss: 0.0009192375582642853
Epoch 4546, Loss: 0.0035157922247890383, Final Batch Loss: 0.00013469011173583567
Epoch 4547, Loss: 0.000444230689026881, Final Batch Loss: 3.7404017348308116e-05
Epoch 4548, Loss: 0.0029139985563233495, Final Batch Loss: 0.002304506953805685
Epoch 4549, Loss: 3.49167239619419e-05, Final Batch Loss: 1.6056317690527067e-05
Epoch 4550, Loss: 1.9167149503118708e-05, Final Batch Loss: 2.3588088424730813e-06
Epoch 4551, Loss: 4.3733472807616636e-05, Final Batch Loss: 9.978119805964525e-07
Epoch 4552, Loss: 0.0010950411087833345, Final Batch Loss: 0.0001336959539912641
Epoch 4553, Loss: 0.009561233415297465, Final Batch Loss: 0.009543673135340214
Epoch 4554, Loss: 0.0001501508304500021, Final Batch Loss: 8.808451821096241e-05
Epoch 4555, Loss: 0.0002342281732126139, Final Batch Loss: 0.0001549176377011463
Epoch 4556, Loss: 3.5659307

Epoch 4662, Loss: 9.922138087858912e-05, Final Batch Loss: 9.33453266043216e-05
Epoch 4663, Loss: 0.001896886590543545, Final Batch Loss: 0.0018952168757095933
Epoch 4664, Loss: 0.005814036710944492, Final Batch Loss: 8.869905286701396e-05
Epoch 4665, Loss: 6.543551990034757e-06, Final Batch Loss: 3.089836582148564e-06
Epoch 4666, Loss: 8.892336154531222e-06, Final Batch Loss: 7.675907909288071e-07
Epoch 4667, Loss: 0.0009720309869862831, Final Batch Loss: 3.709178372446331e-06
Epoch 4668, Loss: 0.00016428346407337813, Final Batch Loss: 8.1257121564704e-06
Epoch 4669, Loss: 0.0005902738678287278, Final Batch Loss: 2.4224766548286425e-06
Epoch 4670, Loss: 3.458009723544819e-05, Final Batch Loss: 3.169697265548166e-06
Epoch 4671, Loss: 0.0003059528489757213, Final Batch Loss: 4.903874469164293e-06
Epoch 4672, Loss: 2.147474447156128e-05, Final Batch Loss: 2.285632263010484e-06
Epoch 4673, Loss: 0.0012504586206887325, Final Batch Loss: 5.995012088533258e-06
Epoch 4674, Loss: 9.44464941312

Epoch 4782, Loss: 6.208292234077817e-06, Final Batch Loss: 5.6256039897562005e-06
Epoch 4783, Loss: 3.7757064887955494e-06, Final Batch Loss: 6.370295864144282e-07
Epoch 4784, Loss: 0.006776141248792555, Final Batch Loss: 0.006771046668291092
Epoch 4785, Loss: 0.00020347180998214753, Final Batch Loss: 0.00020025516278110445
Epoch 4786, Loss: 0.0006623430681429454, Final Batch Loss: 0.0006504393531940877
Epoch 4787, Loss: 1.489039300395234e-05, Final Batch Loss: 1.1652367902570404e-05
Epoch 4788, Loss: 0.00020191139628877863, Final Batch Loss: 8.047773008001968e-05
Epoch 4789, Loss: 6.10669649176998e-05, Final Batch Loss: 2.4312945242854767e-05
Epoch 4790, Loss: 0.0002889435927500017, Final Batch Loss: 0.0002354861790081486
Epoch 4791, Loss: 9.213194607582409e-05, Final Batch Loss: 6.839980778750032e-05
Epoch 4792, Loss: 2.079507112284773e-05, Final Batch Loss: 4.082936811755644e-06
Epoch 4793, Loss: 0.0001182373562187422, Final Batch Loss: 0.00011273029667790979
Epoch 4794, Loss: 0.000

Epoch 4899, Loss: 4.703837021224899e-05, Final Batch Loss: 1.3540814506995957e-05
Epoch 4900, Loss: 0.0002896673686336726, Final Batch Loss: 0.0002116906107403338
Epoch 4901, Loss: 0.003429141390370205, Final Batch Loss: 0.00026980836992152035
Epoch 4902, Loss: 0.001062343477315153, Final Batch Loss: 0.0010553306201472878
Epoch 4903, Loss: 0.00042469343429729633, Final Batch Loss: 2.261608869957854e-06
Epoch 4904, Loss: 8.465130667900667e-05, Final Batch Loss: 7.742521120235324e-06
Epoch 4905, Loss: 2.8538741389638744e-05, Final Batch Loss: 5.307698302203789e-06
Epoch 4906, Loss: 9.126849545282312e-05, Final Batch Loss: 3.7116125895408913e-05
Epoch 4907, Loss: 4.684184204961639e-05, Final Batch Loss: 2.2373340470949188e-05
Epoch 4908, Loss: 7.133427970984485e-06, Final Batch Loss: 1.9791627892118413e-06
Epoch 4909, Loss: 0.00047199251184792956, Final Batch Loss: 0.00046498325536958873
Epoch 4910, Loss: 7.99600820755586e-06, Final Batch Loss: 9.340633368992712e-07
Epoch 4911, Loss: 0.00

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[36  0  0]
 [ 0 36  0]
 [ 0  0 38]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        36
           1    1.00000   1.00000   1.00000        36
           2    1.00000   1.00000   1.00000        38

    accuracy                        1.00000       110
   macro avg    1.00000   1.00000   1.00000       110
weighted avg    1.00000   1.00000   1.00000       110



In [11]:
n_samples = 10

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples * 3)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples * 3)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Group 5_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples * 3) + 1

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9))
fake_labels = np.concatenate((y_1, y_2, y_3))

fake_features = torch.Tensor(fake_features)

In [12]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[30  0  0]
 [ 0 30  0]
 [ 0  0 30]]
              precision    recall  f1-score   support

         0.0    1.00000   1.00000   1.00000        30
         1.0    1.00000   1.00000   1.00000        30
         2.0    1.00000   1.00000   1.00000        30

    accuracy                        1.00000        90
   macro avg    1.00000   1.00000   1.00000        90
weighted avg    1.00000   1.00000   1.00000        90



# User Classifier

In [13]:
activities = [1, 3, 4]
users = [23, 25, 27]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [14]:
for k in range(len(y)):
    if y[k] == 23:
        y[k] = 0
    elif y[k] == 25:
        y[k] = 1
    else:
        y[k] = 2

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [16]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.592713832855225, Final Batch Loss: 2.28945255279541
Epoch 2, Loss: 4.582348823547363, Final Batch Loss: 2.299422025680542
Epoch 3, Loss: 4.5614330768585205, Final Batch Loss: 2.2721903324127197
Epoch 4, Loss: 4.550448894500732, Final Batch Loss: 2.2759528160095215
Epoch 5, Loss: 4.536012172698975, Final Batch Loss: 2.272475481033325
Epoch 6, Loss: 4.5207977294921875, Final Batch Loss: 2.254392385482788
Epoch 7, Loss: 4.505822420120239, Final Batch Loss: 2.243162155151367
Epoch 8, Loss: 4.494362115859985, Final Batch Loss: 2.253983974456787
Epoch 9, Loss: 4.473474740982056, Final Batch Loss: 2.2413506507873535
Epoch 10, Loss: 4.454813241958618, Final Batch Loss: 2.236320972442627
Epoch 11, Loss: 4.430752992630005, Final Batch Loss: 2.2075843811035156
Epoch 12, Loss: 4.40290093421936, Final Batch Loss: 2.1948773860931396
Epoch 13, Loss: 4.373013257980347, Final Batch Loss: 2.1801562309265137
Epoch 14, Loss: 4.340824604034424, Final Batch Loss: 2.177187442779541
Epoch 15,

Epoch 126, Loss: 1.619704782962799, Final Batch Loss: 0.8073249459266663
Epoch 127, Loss: 1.6581289172172546, Final Batch Loss: 0.8561289310455322
Epoch 128, Loss: 1.637282907962799, Final Batch Loss: 0.8151730298995972
Epoch 129, Loss: 1.6364168524742126, Final Batch Loss: 0.7978498935699463
Epoch 130, Loss: 1.6114662289619446, Final Batch Loss: 0.7987414002418518
Epoch 131, Loss: 1.6039546728134155, Final Batch Loss: 0.8285960555076599
Epoch 132, Loss: 1.665042519569397, Final Batch Loss: 0.7762026786804199
Epoch 133, Loss: 1.6698585152626038, Final Batch Loss: 0.8753190040588379
Epoch 134, Loss: 1.5924875140190125, Final Batch Loss: 0.7928985357284546
Epoch 135, Loss: 1.6339794397354126, Final Batch Loss: 0.800942063331604
Epoch 136, Loss: 1.570720374584198, Final Batch Loss: 0.774533212184906
Epoch 137, Loss: 1.631464660167694, Final Batch Loss: 0.8236614465713501
Epoch 138, Loss: 1.5816344618797302, Final Batch Loss: 0.8049893379211426
Epoch 139, Loss: 1.5791565775871277, Final Ba

Epoch 257, Loss: 1.1550130248069763, Final Batch Loss: 0.582808256149292
Epoch 258, Loss: 1.186354398727417, Final Batch Loss: 0.576886773109436
Epoch 259, Loss: 1.1867223381996155, Final Batch Loss: 0.5855450630187988
Epoch 260, Loss: 1.1714747548103333, Final Batch Loss: 0.6144552826881409
Epoch 261, Loss: 1.1534550786018372, Final Batch Loss: 0.555401623249054
Epoch 262, Loss: 1.1798474788665771, Final Batch Loss: 0.5604533553123474
Epoch 263, Loss: 1.1677351593971252, Final Batch Loss: 0.5512164235115051
Epoch 264, Loss: 1.1740731000900269, Final Batch Loss: 0.5718376636505127
Epoch 265, Loss: 1.1792755126953125, Final Batch Loss: 0.6037541031837463
Epoch 266, Loss: 1.133442223072052, Final Batch Loss: 0.5832855105400085
Epoch 267, Loss: 1.1450448632240295, Final Batch Loss: 0.5583735704421997
Epoch 268, Loss: 1.1301632523536682, Final Batch Loss: 0.5652783513069153
Epoch 269, Loss: 1.1367736458778381, Final Batch Loss: 0.5662752389907837
Epoch 270, Loss: 1.1032909750938416, Final 

Epoch 386, Loss: 0.6800519227981567, Final Batch Loss: 0.33290690183639526
Epoch 387, Loss: 0.6919539868831635, Final Batch Loss: 0.34291931986808777
Epoch 388, Loss: 0.689581423997879, Final Batch Loss: 0.36826327443122864
Epoch 389, Loss: 0.70786052942276, Final Batch Loss: 0.3295608162879944
Epoch 390, Loss: 0.6958072185516357, Final Batch Loss: 0.32929444313049316
Epoch 391, Loss: 0.7061509788036346, Final Batch Loss: 0.32320183515548706
Epoch 392, Loss: 0.6731280982494354, Final Batch Loss: 0.31690531969070435
Epoch 393, Loss: 0.6977520287036896, Final Batch Loss: 0.3508240282535553
Epoch 394, Loss: 0.7308638393878937, Final Batch Loss: 0.40050891041755676
Epoch 395, Loss: 0.6776743531227112, Final Batch Loss: 0.34835198521614075
Epoch 396, Loss: 0.6494481563568115, Final Batch Loss: 0.3103172779083252
Epoch 397, Loss: 0.6726797223091125, Final Batch Loss: 0.3054424524307251
Epoch 398, Loss: 0.6509007513523102, Final Batch Loss: 0.2881084978580475
Epoch 399, Loss: 0.61773702502250

Epoch 507, Loss: 0.4072529226541519, Final Batch Loss: 0.18107837438583374
Epoch 508, Loss: 0.4781186878681183, Final Batch Loss: 0.23465536534786224
Epoch 509, Loss: 0.43674229085445404, Final Batch Loss: 0.20279961824417114
Epoch 510, Loss: 0.3932747542858124, Final Batch Loss: 0.19050383567810059
Epoch 511, Loss: 0.4783874601125717, Final Batch Loss: 0.24778127670288086
Epoch 512, Loss: 0.4631339907646179, Final Batch Loss: 0.26228615641593933
Epoch 513, Loss: 0.3649882674217224, Final Batch Loss: 0.18860596418380737
Epoch 514, Loss: 0.3968724459409714, Final Batch Loss: 0.15658840537071228
Epoch 515, Loss: 0.4430171251296997, Final Batch Loss: 0.2415766417980194
Epoch 516, Loss: 0.4349319487810135, Final Batch Loss: 0.2553214728832245
Epoch 517, Loss: 0.41686050593852997, Final Batch Loss: 0.17808488011360168
Epoch 518, Loss: 0.4855312407016754, Final Batch Loss: 0.3041589856147766
Epoch 519, Loss: 0.4620882719755173, Final Batch Loss: 0.24694544076919556
Epoch 520, Loss: 0.3928361

...

Epoch 4503, Loss: 0.08740771934390068, Final Batch Loss: 0.03304858133196831
Epoch 4504, Loss: 0.10327430814504623, Final Batch Loss: 0.04168868809938431
Epoch 4505, Loss: 0.07689854130148888, Final Batch Loss: 0.03411953151226044
Epoch 4506, Loss: 0.10774689167737961, Final Batch Loss: 0.05759118124842644
Epoch 4507, Loss: 0.20811421051621437, Final Batch Loss: 0.16144630312919617
Epoch 4508, Loss: 0.11887453496456146, Final Batch Loss: 0.0479050949215889
Epoch 4509, Loss: 0.11110401153564453, Final Batch Loss: 0.04142434149980545
Epoch 4510, Loss: 0.14872341603040695, Final Batch Loss: 0.0995706245303154
Epoch 4511, Loss: 0.1297181248664856, Final Batch Loss: 0.08241744339466095
Epoch 4512, Loss: 0.12963860481977463, Final Batch Loss: 0.0632023736834526
Epoch 4513, Loss: 0.14432837441563606, Final Batch Loss: 0.05808902904391289
Epoch 4514, Loss: 0.14032452553510666, Final Batch Loss: 0.07086043059825897
Epoch 4515, Loss: 0.11780659854412079, Final Batch Loss: 0.08125478029251099
Epo

Epoch 4622, Loss: 0.09294522181153297, Final Batch Loss: 0.047973860055208206
Epoch 4623, Loss: 0.0927330031991005, Final Batch Loss: 0.048617489635944366
Epoch 4624, Loss: 0.13518749177455902, Final Batch Loss: 0.07215970754623413
Epoch 4625, Loss: 0.09918446466326714, Final Batch Loss: 0.03525276109576225
Epoch 4626, Loss: 0.11014808714389801, Final Batch Loss: 0.05627971142530441
Epoch 4627, Loss: 0.14446235075592995, Final Batch Loss: 0.08704819530248642
Epoch 4628, Loss: 0.12966308742761612, Final Batch Loss: 0.044301584362983704
Epoch 4629, Loss: 0.10047056898474693, Final Batch Loss: 0.03572804108262062
Epoch 4630, Loss: 0.11820574849843979, Final Batch Loss: 0.0619051530957222
Epoch 4631, Loss: 0.08424264192581177, Final Batch Loss: 0.034921031445264816
Epoch 4632, Loss: 0.0845007486641407, Final Batch Loss: 0.03922135382890701
Epoch 4633, Loss: 0.09758774563670158, Final Batch Loss: 0.05525355786085129
Epoch 4634, Loss: 0.10095690563321114, Final Batch Loss: 0.0491704680025577

Epoch 4750, Loss: 0.13907189667224884, Final Batch Loss: 0.06911943852901459
Epoch 4751, Loss: 0.09039301052689552, Final Batch Loss: 0.041532643139362335
Epoch 4752, Loss: 0.08901868760585785, Final Batch Loss: 0.04495299980044365
Epoch 4753, Loss: 0.0864651557058096, Final Batch Loss: 0.02699785865843296
Epoch 4754, Loss: 0.10172628238797188, Final Batch Loss: 0.03774956241250038
Epoch 4755, Loss: 0.0929366908967495, Final Batch Loss: 0.0561516173183918
Epoch 4756, Loss: 0.1410645693540573, Final Batch Loss: 0.04651271551847458
Epoch 4757, Loss: 0.09934911131858826, Final Batch Loss: 0.04794040694832802
Epoch 4758, Loss: 0.10200361907482147, Final Batch Loss: 0.04939728230237961
Epoch 4759, Loss: 0.09400519356131554, Final Batch Loss: 0.047594863921403885
Epoch 4760, Loss: 0.09719153121113777, Final Batch Loss: 0.04554501920938492
Epoch 4761, Loss: 0.10201752558350563, Final Batch Loss: 0.05210988596081734
Epoch 4762, Loss: 0.09349725767970085, Final Batch Loss: 0.05228007212281227
E

Epoch 4872, Loss: 0.09016014263033867, Final Batch Loss: 0.04863593354821205
Epoch 4873, Loss: 0.08907696232199669, Final Batch Loss: 0.019368071109056473
Epoch 4874, Loss: 0.09658830240368843, Final Batch Loss: 0.06251917779445648
Epoch 4875, Loss: 0.08732975460588932, Final Batch Loss: 0.02604779414832592
Epoch 4876, Loss: 0.10135439410805702, Final Batch Loss: 0.05283473804593086
Epoch 4877, Loss: 0.09656842052936554, Final Batch Loss: 0.05809721350669861
Epoch 4878, Loss: 0.10247951000928879, Final Batch Loss: 0.05926154926419258
Epoch 4879, Loss: 0.11462432146072388, Final Batch Loss: 0.0706796646118164
Epoch 4880, Loss: 0.09321613609790802, Final Batch Loss: 0.05537528172135353
Epoch 4881, Loss: 0.08745510876178741, Final Batch Loss: 0.051938582211732864
Epoch 4882, Loss: 0.08816709741950035, Final Batch Loss: 0.05153356119990349
Epoch 4883, Loss: 0.09073716774582863, Final Batch Loss: 0.044584643095731735
Epoch 4884, Loss: 0.08614088222384453, Final Batch Loss: 0.046047072857618

Epoch 4997, Loss: 0.10720906406641006, Final Batch Loss: 0.06332631409168243
Epoch 4998, Loss: 0.1129295825958252, Final Batch Loss: 0.04486106336116791
Epoch 4999, Loss: 0.122499980032444, Final Batch Loss: 0.05389717221260071
Epoch 5000, Loss: 0.10784735903143883, Final Batch Loss: 0.03992145135998726


In [17]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[39  0  0]
 [ 4 38  0]
 [ 1  0 28]]
              precision    recall  f1-score   support

           0    0.88636   1.00000   0.93976        39
           1    1.00000   0.90476   0.95000        42
           2    1.00000   0.96552   0.98246        29

    accuracy                        0.95455       110
   macro avg    0.96212   0.95676   0.95741       110
weighted avg    0.95971   0.95455   0.95493       110



In [18]:
fake_labels = [0] * n_samples + [1] * n_samples + [2] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples
fake_labels = np.asarray(fake_labels)

In [19]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(fake_labels, preds.cpu()))
print(metrics.classification_report(fake_labels, preds.cpu(), digits = 5))

[[30  0  0]
 [ 2 23  5]
 [ 2  5 23]]
              precision    recall  f1-score   support

           0    0.88235   1.00000   0.93750        30
           1    0.82143   0.76667   0.79310        30
           2    0.82143   0.76667   0.79310        30

    accuracy                        0.84444        90
   macro avg    0.84174   0.84444   0.84124        90
weighted avg    0.84174   0.84444   0.84124        90

